In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 8)

from scipy import stats
import collections

import warnings
# warnings.filterwarnings('ignore')
from matplotlib.backends.backend_pdf import PdfPages

from pathlib import Path

import db_queries as db
import vivarium_helpers.id_helper as idh
import gbd_mapping
from vivarium import Artifact

# Add the repo directory vivarium_research_ciff_sam/ to sys.path
import os, sys
repo_path = os.path.abspath('../..')
sys.path.append(repo_path)
# Assumes vivarium_research_ciff_sam/ is in sys.path
import model_validation.vivarium_transformed_output as vto
# import model_validation.vivarium_raw_output as vro
import model_validation.vivarium_output_processing as vop
import model_validation.ciff_sam_results as csr

!pwd
!whoami
!date

/ihme/homes/ndbs/vivarium_research_ciff_sam/model_validation/model4
ndbs
Mon Sep 27 12:28:47 PDT 2021


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
data = csr.VivariumResults.cleaned_from_model_spec(4.1)
data.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'stunting_state_person_time',
 'population',
 'ylls',
 'ylds',
 'cause_state_person_time',
 'cause_transition_count']

In [4]:
data.wasting_state_person_time.scenario.unique()

array(['baseline', 'sqlns', 'wasting_treatment'], dtype=object)

# Investigate bug Rajan identified


From Rajan via Slack on 9/22/2021:
> Just as a heads up, in the partial runs we got yesterday the alternative wasting scenarios doesn't capture any wasting person time or transitions in the covered population due to a bug I found today. The baseline scenario should be fine as well as the other count data.

## Check wasting person-time

There are nonzero values when one of `sqlns` or `wasting_treatment` is `'uncovered'`, but zero person time when both are `'covered'`.

In [5]:
data.wasting_state_person_time

,sex,year,wasting_state,measure,input_draw,scenario,value,sq_lns,wasting_treatment,age
0,female,2022,mild_child_wasting,state_person_time,29,baseline,0.000000,covered,covered,early_neonatal
1,female,2022,mild_child_wasting,state_person_time,29,baseline,0.000000,covered,uncovered,early_neonatal
2,female,2022,mild_child_wasting,state_person_time,29,baseline,201.939767,uncovered,covered,early_neonatal
3,female,2022,mild_child_wasting,state_person_time,29,baseline,207.442847,uncovered,uncovered,early_neonatal
...,...,...,...,...,...,...,...,...,...,...
34556,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,0.000000,covered,covered,2_to_4
34557,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,0.000000,covered,uncovered,2_to_4
34558,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,198117.045859,uncovered,covered,2_to_4
34559,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,21226.153320,uncovered,uncovered,2_to_4


In [6]:
data.wasting_state_person_time.query("sq_lns=='covered' and value>0").wasting_treatment.unique()

array(['covered', 'uncovered'], dtype=object)

In [7]:
data.wasting_state_person_time.query("wasting_treatment=='covered' and value>0").sq_lns.unique()

array(['uncovered', 'covered'], dtype=object)

In [8]:
data.wasting_state_person_time.query("wasting_treatment=='covered' and sq_lns=='covered' and value>0")

,sex,year,wasting_state,measure,input_draw,scenario,value,sq_lns,wasting_treatment,age
3492,female,2023,mild_child_wasting,state_person_time,29,sqlns,9354.731006,covered,covered,6-11_months
3496,female,2023,mild_child_wasting,state_person_time,29,sqlns,19466.928131,covered,covered,12_to_23_months
3500,female,2023,mild_child_wasting,state_person_time,29,sqlns,55741.885010,covered,covered,2_to_4
3564,female,2023,mild_child_wasting,state_person_time,223,sqlns,9158.036961,covered,covered,6-11_months
...,...,...,...,...,...,...,...,...,...,...
34460,male,2026,susceptible_to_child_wasting,state_person_time,829,sqlns,181110.965092,covered,covered,2_to_4
34524,male,2026,susceptible_to_child_wasting,state_person_time,946,sqlns,28868.918549,covered,covered,6-11_months
34528,male,2026,susceptible_to_child_wasting,state_person_time,946,sqlns,58140.585900,covered,covered,12_to_23_months
34532,male,2026,susceptible_to_child_wasting,state_person_time,946,sqlns,179993.210130,covered,covered,2_to_4


In [9]:
data.wasting_state_person_time.query("wasting_treatment=='covered' and value>0").scenario.unique()

array(['baseline', 'sqlns', 'wasting_treatment'], dtype=object)

In [10]:
data.wasting_state_person_time.query("sq_lns=='covered' and value>0").scenario.unique()

array(['sqlns'], dtype=object)

## Check wasting transition counts

Same thing - no person time when covered by both sqlns and wasting_treatment.

In [11]:
data.wasting_transition_count

,sex,year,measure,input_draw,scenario,value,sq_lns,wasting_treatment,age,transition,from_state,to_state
0,female,2022,transition_count,29,baseline,0.0,covered,covered,early_neonatal,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition
1,female,2022,transition_count,29,baseline,0.0,covered,uncovered,early_neonatal,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition
2,female,2022,transition_count,29,baseline,0.0,uncovered,covered,early_neonatal,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition
3,female,2022,transition_count,29,baseline,0.0,uncovered,uncovered,early_neonatal,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition
...,...,...,...,...,...,...,...,...,...,...,...,...
60476,male,2026,transition_count,946,wasting_treatment,0.0,covered,covered,2_to_4,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting
60477,male,2026,transition_count,946,wasting_treatment,0.0,covered,uncovered,2_to_4,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting
60478,male,2026,transition_count,946,wasting_treatment,21109.0,uncovered,covered,2_to_4,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting
60479,male,2026,transition_count,946,wasting_treatment,2245.0,uncovered,uncovered,2_to_4,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting


In [12]:
data.wasting_transition_count.query("wasting_treatment=='covered' and sq_lns=='covered' and value>0")

,sex,year,measure,input_draw,scenario,value,sq_lns,wasting_treatment,age,transition,from_state,to_state
6084,female,2023,transition_count,29,sqlns,21215.0,covered,covered,6-11_months,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition
6088,female,2023,transition_count,29,sqlns,39111.0,covered,covered,12_to_23_months,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition
6092,female,2023,transition_count,29,sqlns,93310.0,covered,covered,2_to_4,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition
6156,female,2023,transition_count,223,sqlns,23268.0,covered,covered,6-11_months,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition
...,...,...,...,...,...,...,...,...,...,...,...,...
60380,male,2026,transition_count,829,sqlns,19432.0,covered,covered,2_to_4,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting
60444,male,2026,transition_count,946,sqlns,3476.0,covered,covered,6-11_months,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting
60448,male,2026,transition_count,946,sqlns,7284.0,covered,covered,12_to_23_months,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting
60452,male,2026,transition_count,946,sqlns,19174.0,covered,covered,2_to_4,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting


# Check that 3 methods of computing wasting prevalence give the same result

Yes, they do! My functions are commutative, woo hoo!

In [13]:
data.compute_total_person_time()

In [14]:
pt_by_wasting_state = vop.stratify(
    data.wasting_state_person_time.query("year > '2022'"), ['wasting_state'])
pt_by_wasting_state

,wasting_state,input_draw,scenario,value
0,mild_child_wasting,29,baseline,8.971736e+05
1,mild_child_wasting,29,sqlns,9.688046e+05
2,mild_child_wasting,29,wasting_treatment,9.373038e+05
3,mild_child_wasting,223,baseline,8.954353e+05
...,...,...,...,...
140,susceptible_to_child_wasting,829,wasting_treatment,2.903149e+06
141,susceptible_to_child_wasting,946,baseline,2.860194e+06
142,susceptible_to_child_wasting,946,sqlns,2.894737e+06
143,susceptible_to_child_wasting,946,wasting_treatment,2.882659e+06


In [15]:
pt = vop.stratify(data.person_time.query("age != 'all_ages' and year > '2022'"), [])
pt

,input_draw,scenario,value
0,29,baseline,4.178410e+06
1,29,sqlns,4.179728e+06
2,29,wasting_treatment,4.179065e+06
3,223,baseline,4.162766e+06
...,...,...,...
32,829,wasting_treatment,4.208040e+06
33,946,baseline,4.173020e+06
34,946,sqlns,4.173859e+06
35,946,wasting_treatment,4.173466e+06


In [16]:
pop_fraction_by_wasting_state = vop.ratio(
    pt_by_wasting_state,
    pt,
    strata = [],
    numerator_broadcast='wasting_state',
    record_inputs=False
)
pop_fraction_by_wasting_state

,input_draw,scenario,wasting_state,value
0,29,baseline,mild_child_wasting,0.214716
1,29,baseline,moderate_acute_malnutrition,0.080046
2,29,baseline,severe_acute_malnutrition,0.019478
3,29,baseline,susceptible_to_child_wasting,0.685760
...,...,...,...,...
140,946,wasting_treatment,mild_child_wasting,0.223208
141,946,wasting_treatment,moderate_acute_malnutrition,0.069967
142,946,wasting_treatment,severe_acute_malnutrition,0.016114
143,946,wasting_treatment,susceptible_to_child_wasting,0.690711


In [17]:
pop_fraction_by_wasting_state2 = vop.value(pt_by_wasting_state) / vop.value(pt)
pop_fraction_by_wasting_state2

value
input_draw scenario          wasting_state                         
29         baseline          mild_child_wasting            0.214716
                             moderate_acute_malnutrition   0.080046
                             severe_acute_malnutrition     0.019478
                             susceptible_to_child_wasting  0.685760
...                                                             ...
946        wasting_treatment mild_child_wasting            0.223208
                             moderate_acute_malnutrition   0.069967
                             severe_acute_malnutrition     0.016114
                             susceptible_to_child_wasting  0.690711

[144 rows x 1 columns]

In [18]:
vop.value(pop_fraction_by_wasting_state2)

value
input_draw scenario          wasting_state                         
29         baseline          mild_child_wasting            0.214716
                             moderate_acute_malnutrition   0.080046
                             severe_acute_malnutrition     0.019478
                             susceptible_to_child_wasting  0.685760
...                                                             ...
946        wasting_treatment mild_child_wasting            0.223208
                             moderate_acute_malnutrition   0.069967
                             severe_acute_malnutrition     0.016114
                             susceptible_to_child_wasting  0.690711

[144 rows x 1 columns]

In [19]:
vop.assert_values_equal(pop_fraction_by_wasting_state, pop_fraction_by_wasting_state2)

In [20]:
pop_fraction_by_wasting_state3 = vop.ratio(
    data.wasting_state_person_time.query("year > '2022'"),
    data.person_time.query("age != 'all_ages' and year > '2022'"),
    strata = [],
    numerator_broadcast='wasting_state',
    record_inputs=False
)
pop_fraction_by_wasting_state3

,input_draw,scenario,wasting_state,value
0,29,baseline,mild_child_wasting,0.214716
1,29,baseline,moderate_acute_malnutrition,0.080046
2,29,baseline,severe_acute_malnutrition,0.019478
3,29,baseline,susceptible_to_child_wasting,0.685760
...,...,...,...,...
140,946,wasting_treatment,mild_child_wasting,0.223208
141,946,wasting_treatment,moderate_acute_malnutrition,0.069967
142,946,wasting_treatment,severe_acute_malnutrition,0.016114
143,946,wasting_treatment,susceptible_to_child_wasting,0.690711


In [21]:
vop.assert_values_equal(pop_fraction_by_wasting_state, pop_fraction_by_wasting_state3)

# Create a small dataframe for testing

In [22]:
df = pop_fraction_by_wasting_state.loc[:5]
df

,input_draw,scenario,wasting_state,value
0,29,baseline,mild_child_wasting,0.214716
1,29,baseline,moderate_acute_malnutrition,0.080046
2,29,baseline,severe_acute_malnutrition,0.019478
3,29,baseline,susceptible_to_child_wasting,0.685760
4,29,sqlns,mild_child_wasting,0.231787
5,29,sqlns,moderate_acute_malnutrition,0.060256


In [23]:
df.set_index('wasting_state').reset_index()

,wasting_state,input_draw,scenario,value
0,mild_child_wasting,29,baseline,0.214716
1,moderate_acute_malnutrition,29,baseline,0.080046
2,severe_acute_malnutrition,29,baseline,0.019478
3,susceptible_to_child_wasting,29,baseline,0.685760
4,mild_child_wasting,29,sqlns,0.231787
5,moderate_acute_malnutrition,29,sqlns,0.060256


In [24]:
vop.value(df.set_index('wasting_state'))

value
input_draw scenario          
29         baseline  0.214716
           baseline  0.080046
           baseline  0.019478
           baseline  0.685760
           sqlns     0.231787
           sqlns     0.060256

In [25]:
vop.value(df.set_index('wasting_state'), include='wasting_state')

,,,value
wasting_state,input_draw,scenario,
mild_child_wasting,29,baseline,0.214716
moderate_acute_malnutrition,29,baseline,0.080046
severe_acute_malnutrition,29,baseline,0.019478
susceptible_to_child_wasting,29,baseline,0.685760
mild_child_wasting,29,sqlns,0.231787
moderate_acute_malnutrition,29,sqlns,0.060256


In [26]:
vop.value(df, exclude='input_draw')

value
scenario wasting_state                         
baseline mild_child_wasting            0.214716
         moderate_acute_malnutrition   0.080046
         severe_acute_malnutrition     0.019478
         susceptible_to_child_wasting  0.685760
sqlns    mild_child_wasting            0.231787
         moderate_acute_malnutrition   0.060256

In [27]:
vop.value(df.set_index('input_draw'), exclude='input_draw')

value
scenario wasting_state                         
baseline mild_child_wasting            0.214716
         moderate_acute_malnutrition   0.080046
         severe_acute_malnutrition     0.019478
         susceptible_to_child_wasting  0.685760
sqlns    mild_child_wasting            0.231787
         moderate_acute_malnutrition   0.060256

In [28]:
temp = vop._ensure_columns_not_levels(df, vop.list_columns('input_draw'))
temp

,input_draw,scenario,wasting_state,value
0,29,baseline,mild_child_wasting,0.214716
1,29,baseline,moderate_acute_malnutrition,0.080046
2,29,baseline,severe_acute_malnutrition,0.019478
3,29,baseline,susceptible_to_child_wasting,0.685760
4,29,sqlns,mild_child_wasting,0.231787
5,29,sqlns,moderate_acute_malnutrition,0.060256


In [29]:
vop.list_columns(None, 'input_draw', 'value')

[None, 'input_draw', 'value']

In [30]:
vop._ensure_columns_not_levels(df, vop.list_columns(None, 'input_draw', 'value'))

,index,input_draw,scenario,wasting_state,value
0,0,29,baseline,mild_child_wasting,0.214716
1,1,29,baseline,moderate_acute_malnutrition,0.080046
2,2,29,baseline,severe_acute_malnutrition,0.019478
3,3,29,baseline,susceptible_to_child_wasting,0.685760
4,4,29,sqlns,mild_child_wasting,0.231787
5,5,29,sqlns,moderate_acute_malnutrition,0.060256


In [31]:
vop.list_columns(None, 'input_draw', 'value', default=[])

['input_draw', 'value']